In [68]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn import metrics
from sklearn.decomposition import PCA

In [69]:
#reading data
data = pd.read_csv('EOCC_New_data_with_zero_updated_columns.csv')

In [70]:
#pca
pca1 = PCA(n_components=1)
pc_w_m = principalComponents = pca1.fit_transform(data[['Mileage_since_last _failure','weeks_in_service']])
data['PCA_1']=pc_w_m
#data = data.drop(['Mileage_since_last _failure','weeks_in_service'],axis = 1)

In [71]:
data['CAR_CLASS'] = pd.factorize(data.CAR_CLASS)[0]
data['EQUIP_FEAT_DESC'] = pd.factorize(data.EQUIP_FEAT_DESC)[0]
data['car_length'] = pd.factorize(data.car_length)[0]
data['car_tonnage'] = pd.factorize(data.car_tonnage)[0]
data['BUILDER_NAME'] = pd.factorize(data.BUILDER_NAME)[0]

In [72]:
best_vals_reg=[]
best_vals_reg_500=[]

for j in range(15):
    data1 = pd.DataFrame(data['CAR NUMBER'].unique())
    data1.columns=['CAR NUMBER']
    data2=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    data2.columns=['RandomNum']
    data1['RandomNum']=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    data_w_rand=pd.merge(data,data1,on='CAR NUMBER',how='left')

    data_new =data_w_rand.drop(['CAR NUMBER','key','LOCATION','RESPONSE CODE','WHY MADE','PART APPLIED DATE','UPDATED PART REPAIR DATE','UPDATED PART REPAIR DATE','BUILT DATE'],axis =1)

    data_500 = data_new[(data_new['Mileage_since_last _failure']< 500000) | (data_new['Mileage_since_last _failure']> 650000)]
    
    print(len(data_new))
    print(len(data_500))
    #train and test  
    data_train = data_new[data_new['RandomNum'] <= 0.75]
    data_test = data_new[data_new['RandomNum'] > 0.75]

    y_train = data_train['TARGET']
    x_train = data_train.drop(['TARGET'],axis =1)
    y_test = data_test['TARGET']
    x_test = data_test.drop(['TARGET'],axis =1)

    clf = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=5000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20,                             
                                    n_jobs = -1).fit(x_train,y_train)
    vals=[]
    for i in np.linspace(0.01,0.2,50):
        predicted_probas = clf.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals.append(metrics.auc(fpr, tpr))
    print(max(vals))
    best_vals_reg.append(max(vals)) 
    
        #train and test  
    data_train = data_500[data_500['RandomNum'] <= 0.75]
    data_test = data_500[data_500['RandomNum'] > 0.75]

    y_train = data_train['TARGET']
    x_train = data_train.drop(['TARGET'],axis =1)
    y_test = data_test['TARGET']
    x_test = data_test.drop(['TARGET'],axis =1)

    clf = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=5000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20,                             
                                    n_jobs = -1).fit(x_train,y_train)
    vals=[]
    for i in np.linspace(0.01,0.2,50):
        predicted_probas = clf.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals.append(metrics.auc(fpr, tpr))
    print(max(vals))
    best_vals_reg_500.append(max(vals)) 

21979
17135
0.698154807636
0.710638794319
21979
17135
0.645970163258
0.646286025596
21979
17135
0.705944112689
0.688446668105
21979
17135
0.640462172568
0.639534707027
21979
17135
0.665666404802
0.671145880833
21979
17135
0.652448392138
0.671387316082
21979
17135
0.673534722376
0.659909295816
21979
17135
0.66525963864
0.683276710159
21979
17135
0.642002237136
0.661332860967
21979
17135
0.676957813393
0.672340445933
21979
17135
0.68822191748
0.665424887027
21979
17135
0.651070284575
0.685544900804
21979
17135
0.641621515313
0.647899159664
21979
17135
0.667901293017
0.668249648806
21979
17135
0.656809501901
0.652096216969


In [73]:
np.mean(best_vals_reg)

0.66480166512812999

In [74]:
np.mean( best_vals_reg_500)

0.6682342345404767